<span style="color: #FF6D6B">

#### Resources
</span>

- [Link to source - Cohere Docs](https://docs.cohere.com/docs/validating-outputs#wrap-an-llm-call-with-the-guard-object)

<span style="color: #FF6D6B">

#### Dependencies
</span>

In [ ]:
# pip install cohere guardrails-ai -q
# !guardrails hub install hub://guardrails/valid_range
# !guardrails hub install hub://guardrails/valid_choices

<span style="color: #FF6D6B">

#### Imports
</span>

In [ ]:
import cohere
import guardrails as gd
from guardrails.hub import ValidRange, ValidChoices
from pydantic import BaseModel, Field
from rich import print
from typing import List

<span style="color: #FF6D6B">

#### Create a Cohere Client
</span>

In [ ]:
co = cohere.Client(api_key="COHERE_API_KEY")

<span style="color: #FF6D6B">

#### Create a Pydantic class to validate
</span>

<span style="color: #30AADD">

##### Simple Class
</span>

In [ ]:
class Overview(BaseModel):
    topic: str = Field(..., description="Identify the main topic that the user is asking about", validators=[ValidChoices(topics,on_fail="reask")])

<span style="color: #30AADD">

##### Nested Class
</span>

In [ ]:
class Topics(BaseModel):
        topic: str = Field(..., description=f"{subtopic_description_final}", validators=[ValidChoices(final_list_of_subtopics,on_fail="reask")])
        subTopics: List[str] = Field(..., description=f"Labels for the topic. For example, {labels_description_final}")
        others_tagging: str = Field(..., description="If the topic is **Others**, give a two word description of the topic. Otherwise, write NA.")
    

    class Overview(BaseModel):
        title: str = Field(..., description="Title of the content, keep it concise and within 1 line. It should be a statement that helps the user easily identify their conversations between a large language model.")
        shortSummary: str = Field(..., description="2 line description of the overall content covering the keys points in the users query and the solution given by a large language model.")
        detailedSummary: str = Field(..., description="Detailed description of the overall content within 500 words covering the keys points in the users query and the solution given by a large language model.")
        topics: List[Topics] = Field(..., description=f"Identify the primary topic within the theme - {theme} ({theme_description_final}). Only focus on the question asked by the user and the topics it covers.")

<span style="color: #FF6D6B">

#### Define the Prompt
</span>

In [ ]:
PROMPT = """Given the following conversation between a user and an answer engine,
please identify the topic the users query belongs to and return a dictionary with the following:

${convo}

${gr.complete_json_suffix_v2}
"""

<span style="color: #FF6D6B">

#### Create a Guard object
</span>

In [ ]:
guard = gd.Guard.from_pydantic(Overview, prompt=PROMPT)

<span style="color: #FF6D6B">

#### Invoke the Cohere Client
</span>

In [ ]:
response = guard(
    co.chat,
    prompt_params={"convo": text},
    model='command-r-plus',
    temperature=0,
)